In [47]:
# Use "jupyter_plotly_dash" for Apporto.
# Use "jupyter_dash" for local machine.
#from jupyter_plotly_dash import JupyterDash
from jupyter_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import base64


from Animal_Shelter import AnimalShelter

# initialize shelter for database queries
username = "aacuser"
password = "userPass"
shelter = AnimalShelter(username, password)

####################
# Helper Functions #
####################
def dfFormat(inFrame):
    """
    Formats the dataframe so that it has the proper columns, headers, order, etc.

    :param inFrame: Dataframe

    :return: Dataframe
    """
    inFrame.sort_values(by=['animal_id'], inplace=True)
    inFrame = inFrame[["animal_id", "breed", "name", "sex_upon_outcome", "age_upon_outcome_in_weeks", "location_lat", "location_long"]]
    inFrame.age_upon_outcome_in_weeks = inFrame.age_upon_outcome_in_weeks.round()
    inFrame.rename(columns = {'animal_id':'Animal ID', 'breed':'Breed', 'name':'Name', 'sex_upon_outcome':'Outcome Sex',
                                  'age_upon_outcome_in_weeks':'Outcome Age', 'location_lat':'Latitude', 'location_long':'Longitude'}, inplace = True)
    return inFrame

def filterOutput(filterOpt):
    if filterOpt == "waterDog":
        outputFrame = pd.DataFrame.from_records(shelter.readAll(
            {'animal_type': 'Dog',
             'breed': {'$in': ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]},
             'sex_upon_outcome': 'Intact Female',
             'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
             }))
    elif filterOpt == "mtnWldDog":
        outputFrame = pd.DataFrame.from_records(shelter.readAll(
            {'animal_type': 'Dog',
             'breed': {'$in': ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]},
             'sex_upon_outcome': 'Intact Male',
             'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156}
             }))
    elif filterOpt == "disTrkgDog":
        outputFrame = pd.DataFrame.from_records(shelter.readAll(
            {'animal_type': 'Dog',
             'breed': {'$in': ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]},
             'sex_upon_outcome': 'Intact Male',
             'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300}
             }))
    else:
        outputFrame = pd.DataFrame.from_records(shelter.read({'animal_type': 'Dog'}, True))

    return dfFormat(outputFrame)



#############################
# Data Manipulation / Model #
#############################
df = filterOutput(None)

###########################
# Dashboard Layout / View #
###########################
app = JupyterDash('SimpleExample')

# logo for Grazioso Salvare company
image_filename = 'logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div([                      # main div for entire header
        html.Div(                   # div for logo
            html.A(                 # makes logo a hyperlink
                html.Img(id='customer-image',
                    src='data:image/png;base64,{}'.format(encoded_image.decode()),
                    alt='customer image',
                    style={'display':'flex', 'max-width': '100%'}),
                href='https://www.snhu.edu/', target="_blank",
                style={'display':'flex', 'max-width': '200px', 'max-height': '100%'}),
            style={'display':'flex', 'max-width': '200px', 'max-height': '100%'}),

        # div for the title
        html.Div(html.B(html.H1('Service Dog Finder'))),

        # div for author
        html.Div(html.B(html.H5('Dashboard Created By: Preston Burkhardt')),
                 style={'display':'flex', 'align-self': 'flex-end'})
    ],
    # style information for the formatting of the header div
    style={'display':'flex', 'flex-direction': 'row', 'justify-content': 'space-between', 'justify-items':'center', 'align-items':'center', 'width': '100%', 'height': '10rem'}),

    html.Hr(),

    # radio item/filter for displaying certain categories of animals
    html.Div(
        dcc.RadioItems(
            id='SnRtype',
            # sets the options and the values they correspond to
            options=[
                {'label': 'All', 'value': 'allDogs'},
                {'label': 'Water Rescue', 'value': 'waterDog'},
                {'label': 'Mountain/Wilderness Rescue', 'value': 'mtnWldDog'},
                {'label': 'Disaster/Individual Tracking', 'value': 'disTrkgDog'}
            ],
            value='allDogs',
            inline=True,
            style={'display': 'flex', 'justify-content': 'space-evenly'})),

    html.Hr(),

    #data table formatting options
    dt.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        style_cell={'textAlign': 'left'},
        editable=False,
        #filter_action="native",
        #sort_mode="multi",
        #column_selectable='multi',
        row_selectable='single',
        row_deletable=False,
        #selected_columns=[],
        selected_rows=[],
        derived_viewport_selected_rows=[],
        derived_virtual_selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
        style_data={
        'width': '150px', 'minWidth': '150px', 'maxWidth': '150px',
        'overflow': 'hidden',
        'textOverflow': 'ellipsis',}
    ),

    html.Br(),
    html.Hr(),

    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex', 'justify-content': 'space-evenly'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ]),
])

###############################################
# Interaction Between Components / Controller #
###############################################

# Changes the table based on filters in filterOutput() function.
# Based on the values in the radio item/menu.
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('SnRtype', 'value')])
def update_dashboard(filter_type):
    df = filterOutput(filter_type)
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
    return data, columns

# Callback for highlighting table rows that have been selected.
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'derived_viewport_selected_rows')]
)
def update_styles(selected_rows):
    return [{'if': { 'row_index': i }, 'background_color': '#D2F3FF'} for i in selected_rows]

# Callback for displaying/updating the pie chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('SnRtype', 'value')])
def update_graphs(viewData, filterValue):
    #modify dataframe to get a count of all the different breeds
    dFrame = pd.DataFrame(viewData)
    dFrame = dFrame['Breed'].value_counts()
    dFrame = dFrame.reset_index()
    dFrame.columns = ['Breed', 'Total']
    if filterValue == 'allDogs':    #group low counts for the "all dogs" filter option
        dFrame.loc[dFrame['Total'] < 50, 'Breed'] = 'Other Breeds'
    fig = px.pie(dFrame, names='Breed', values='Total', title='Dog Breeds', template='plotly_white')
    fig.update_layout(margin=dict(t=30, b=30, l=30, r=30))
    return [
       dcc.Graph(
           figure=fig,
           style={'width': '500px', 'height': '500px'}
       )
    ]

#This callback handles the map. Marker is displayed for the animal (row) that is selected.
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     Input('datatable-id', "derived_viewport_selected_rows")])
def update_map(viewData, selectedRow):
    #handles index errors by displaying map with no markers
    try:
        dff = pd.DataFrame.from_dict(viewData)
        selectedRow = selectedRow[0]
    except IndexError:
        return [
        dl.Map(style={'width': '500px', 'height': '500px'}, center=[30.75,-97.48], zoom=8,
               children=[dl.TileLayer(id="base-layer-id")])]

    # map marker changes based on which animal (row) is selected
    return [
        dl.Map(style={'width': '500px', 'height': '500px'}, center=[dff.iloc[selectedRow,5],dff.iloc[selectedRow,6]], zoom=8, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.iloc[selectedRow,5],dff.iloc[selectedRow,6]], children=[
                dl.Tooltip(dff.iloc[selectedRow,1]),
                dl.Popup([
                    html.H3("Animal ID:"),
                    html.H5(dff.iloc[selectedRow,0]),
                    html.H3("Animal Name:"),
                    html.H5(dff.iloc[selectedRow,2])
                ])
            ])
        ])
    ]



if __name__ == '__main__':
    # Use "app" for Apporto.
    # Use "app.run_server()" for local machine.
    #app
    app.run_server(host="localhost",port=8051)

C:\Users\prest\AppData\Local\Temp\ipykernel_20492\195423831.py:42: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\prest\AppData\Local\Temp\ipykernel_20492\195423831.py:43: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Dash app running on http://localhost:8051/
